In [1]:
# Importing necessary libraries
import nltk
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, explode
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.types import DoubleType, ArrayType, StringType
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import col, udf
from nltk.sentiment import SentimentIntensityAnalyzer
from pyspark.sql.types import DoubleType, ArrayType, StringType
from pyspark.sql.types import DoubleType
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import explode
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dropout
from pyspark.sql import functions as F

[nltk_data] Downloading package punkt to /home/hduser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/hduser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# Importing PySpark
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Read CSV") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "8g") \
    .getOrCreate()

24/04/02 23:30:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [7]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [8]:
# Read the CSV file into a DataFrame
data_df = spark.read.load("hdfs://localhost:9000/user1/train.csv", format="csv", header=True)

In [9]:
data_df.show(3)

+-----------+--------------------+--------------------+
|class_index|        review_title|         review_text|
+-----------+--------------------+--------------------+
|          3|  more like funchuck|"Gave this to my ...|
|          5|           Inspiring|I hope a lot of p...|
|          5|The best soundtra...|I'm reading a lot...|
+-----------+--------------------+--------------------+
only showing top 3 rows



In [5]:
# Check the number of rows
num_rows = data_df.count()
print("Number of rows:", num_rows)

# Check the list of column names
column_names = data_df.columns
print("Column names:", column_names)


Number of rows: 3000000
Column names: ['class_index', 'review_title', 'review_text']


In [6]:
#Printing Schema
print("Schema:")
data_df.printSchema()

Schema:
root
 |-- class_index: string (nullable = true)
 |-- review_title: string (nullable = true)
 |-- review_text: string (nullable = true)



In [7]:
# Show sample rows after removing null values
data_df.show()

+-----------+--------------------+--------------------+
|class_index|        review_title|         review_text|
+-----------+--------------------+--------------------+
|          3|  more like funchuck|"Gave this to my ...|
|          5|           Inspiring|I hope a lot of p...|
|          5|The best soundtra...|I'm reading a lot...|
|          4|    Chrono Cross OST|"The music of Yas...|
|          5| Too good to be true|Probably the grea...|
|          5|There's a reason ...|There's a reason ...|
|          1|        Buyer beware|"This is a self-p...|
|          4|Errors, but great...|I was a dissapoin...|
|          1|          The Worst!|A complete waste ...|
|          1|           Oh please|I guess you have ...|
|          1|Awful beyond belief!|"I feel I have to...|
|          4|A romantic zen ba...|"When you hear fo...|
|          5|Lower leg comfort...|Excellent stockin...|
|          3|Delivery was very...|It took almost 3 ...|
|          2|sizes recomended ...|sizes are much

In [8]:
# Group by 'class_index' column and perform aggregation
aggregated_df = data_df.groupBy("class_index").agg(
    F.count("*").alias("count_reviews"),
    F.avg("class_index").alias("avg_class_index")
)

In [9]:
# Show the aggregated DataFrame
aggregated_df.show()

+-----------+-------------+---------------+
|class_index|count_reviews|avg_class_index|
+-----------+-------------+---------------+
|          3|       600000|            3.0|
|          5|       600000|            5.0|
|          1|       600000|            1.0|
|          4|       600000|            4.0|
|          2|       600000|            2.0|
+-----------+-------------+---------------+



In [10]:
# Sorting the DataFrame by a single column
sorted_df = data_df.orderBy("class_index")

In [11]:
# Showing the sorted DataFrame
sorted_df.show()

+-----------+--------------------+--------------------+
|class_index|        review_title|         review_text|
+-----------+--------------------+--------------------+
|          1|  Read Dante Instead|Reading this book...|
|          1|         Didn't work|This tape was imm...|
|          1|        unhappy feet|A disappointment ...|
|          1|                Poor|"This is actually...|
|          1|Really? Magic gecko?|Right about the t...|
|          1|             Useless|I don't think it ...|
|          1|             Bad Buy|I bought this rin...|
|          1|  Poor Sound Quality|The sound quality...|
|          1|Shouldn't be adve...|I gave this toy t...|
|          1|     Waste of money.|This thing is a p...|
|          1|     A Steaming Pile|"This album is ho...|
|          1|          Wrong item|Ordered the Belki...|
|          1|     TOTALLY USELESS|"Not only is the ...|
|          1|Product did not work|This didn't work ...|
|          1|Statistics for st...|This book will

In [12]:
# Sort in descending order, using desc() function
sorted_desc_df = data_df.orderBy(data_df["class_index"].desc())
sorted_desc_df.show()

+-----------+--------------------+--------------------+
|class_index|        review_title|         review_text|
+-----------+--------------------+--------------------+
|          5|For those of us w...|"Louis adventures...|
|          5|   Coolest game eva!|I love the nancy ...|
|          5|    Soft and Cuddly!|My son loves his ...|
|          5|      Kitchen Review|My husband and I ...|
|          5|           excellent|Exactly what I no...|
|          5|Kind of creepy bu...|"In the begining ...|
|          5|A Reference Recor...|In the mid '70s, ...|
|          5|great product at ...|I have purchased ...|
|          5|      The New Empire|From the other st...|
|          5|          Great pan!|This is a great q...|
|          5|Everything a roma...|Just finished thi...|
|          5|this is the one y...|best value in a t...|
|          5|               KPT 5|I have KPT 3,5,6 ...|
|          5|  Norpro popover pan|We had popovers b...|
|          5|       giving it all|Real life lyri

In [13]:
# Sorting by multiple columns
multi_sorted_df = data_df.orderBy("class_index", "review_title")
multi_sorted_df.show()

+-----------+--------------------+--------------------+
|class_index|        review_title|         review_text|
+-----------+--------------------+--------------------+
|          1|                null|"I remember order...|
|          1|                null|"I was extremely ...|
|          1|                null|Just wanted to vo...|
|          1|                null|Can't write a rev...|
|          1|                null|"Some good rockin...|
|          1|                null|I didn't really c...|
|          1|                null|These guys sure h...|
|          1|                null|I ordered a 12 fo...|
|          1|                null|"Unless you're a ...|
|          1|                null|This is lightweig...|
|          1|                null|And here we have ...|
|          1|                null|The battery was p...|
|          1|                   !|"This book is com...|
|          1|! - STAY AWAY! WO...|Please for the lo...|
|          1|! CAUTION - NOT T...|!! CAUTION!!Re

In [14]:
# Filtering rows where class_index is equal to 5
filtered_df = data_df.filter(data_df["class_index"] == 5)

In [15]:
# print filtered data
print("Filtered Data:")
filtered_df.show()

Filtered Data:
+-----------+--------------------+--------------------+
|class_index|        review_title|         review_text|
+-----------+--------------------+--------------------+
|          5|           Inspiring|I hope a lot of p...|
|          5|The best soundtra...|I'm reading a lot...|
|          5| Too good to be true|Probably the grea...|
|          5|There's a reason ...|There's a reason ...|
|          5|Lower leg comfort...|Excellent stockin...|
|          5|CHECK OUT THE K60...|Greetings. Kodak ...|
|          5|                ahhh|These will more t...|
|          5|   Excellent Product|This is an excell...|
|          5|"To ""Disappointe...|"You said ""...bu...|
|          5|Awesome - wanna u...|It's just awesome...|
|          5|"Very Happy - Has...|"I got this machi...|
|          5|This is a very go...|Many useful conce...|
|          5|Great reference b...|It seems somebody...|
|          5|Here is a little ...|After you watch a...|
|          5|Rochelle explains...

In [16]:
data_df.show(3)

+-----------+--------------------+--------------------+
|class_index|        review_title|         review_text|
+-----------+--------------------+--------------------+
|          3|  more like funchuck|"Gave this to my ...|
|          5|           Inspiring|I hope a lot of p...|
|          5|The best soundtra...|I'm reading a lot...|
+-----------+--------------------+--------------------+
only showing top 3 rows



In [17]:
from pyspark.sql.functions import col, count

# Count null values in each column
null_counts = data_df.agg(*[count(col(c).isNull().cast("int")).alias(c) for c in data_df.columns])

# Show the null counts
null_counts.show()


+-----------+------------+-----------+
|class_index|review_title|review_text|
+-----------+------------+-----------+
|    3000000|     3000000|    3000000|
+-----------+------------+-----------+



In [18]:
# Show a sample of the DataFrame
data_df.show(5)

+-----------+--------------------+--------------------+
|class_index|        review_title|         review_text|
+-----------+--------------------+--------------------+
|          3|  more like funchuck|"Gave this to my ...|
|          5|           Inspiring|I hope a lot of p...|
|          5|The best soundtra...|I'm reading a lot...|
|          4|    Chrono Cross OST|"The music of Yas...|
|          5| Too good to be true|Probably the grea...|
+-----------+--------------------+--------------------+
only showing top 5 rows



In [19]:
from pyspark.sql.functions import col

# Filter for rows where review_title or review_text is empty
empty_title_df = data_df.filter(col("review_title") == "")
empty_text_df = data_df.filter(col("review_text") == "")

# Count the number of rows with empty review_title and review_text
empty_title_count = empty_title_df.count()
empty_text_count = empty_text_df.count()

print("Number of rows with empty review_title:", empty_title_count)
print("Number of rows with empty review_text:", empty_text_count)


Number of rows with empty review_title: 0
Number of rows with empty review_text: 0


In [20]:
from pyspark.sql.functions import concat_ws

# Concatenate review_title and review_text into a new column named text
data_df = data_df.withColumn('text', concat_ws(' ', data_df['review_title'], data_df['review_text']))
data_df.show()

+-----------+--------------------+--------------------+--------------------+
|class_index|        review_title|         review_text|                text|
+-----------+--------------------+--------------------+--------------------+
|          3|  more like funchuck|"Gave this to my ...|more like funchuc...|
|          5|           Inspiring|I hope a lot of p...|Inspiring I hope ...|
|          5|The best soundtra...|I'm reading a lot...|The best soundtra...|
|          4|    Chrono Cross OST|"The music of Yas...|Chrono Cross OST ...|
|          5| Too good to be true|Probably the grea...|Too good to be tr...|
|          5|There's a reason ...|There's a reason ...|There's a reason ...|
|          1|        Buyer beware|"This is a self-p...|Buyer beware "Thi...|
|          4|Errors, but great...|I was a dissapoin...|Errors, but great...|
|          1|          The Worst!|A complete waste ...|The Worst! A comp...|
|          1|           Oh please|I guess you have ...|Oh please I guess...|